# Trabalho Programação Linear 

## Leitura dos dados 

In [1]:
from pandas_ods_reader import read_ods

df = read_ods("variaveis_decisao.ods", 1)
df

,Item,Unidade,Local transporte,Variável de decisão,Volume [cm^3],Venda (R$),Compra,Lucro,Demanda
0,Ovo de codorna,Dúzia,Kombi,x1,560.0,5.00,3.00,2.00,75.0
1,Ovo Branco,Dúzia,Kombi,x2,1458.0,5.00,3.50,1.50,100.0
2,Ovo Vermelho,Dúzia,Kombi,x3,1458.0,7.00,5.00,2.00,40.0
3,Coxa de Frango,Embalagem,Geladeira,x4,640.0,12.90,7.00,5.90,15.0
4,Picanha Suína,Embalagem,Geladeira,x5,630.0,26.90,18.80,8.10,4.0
5,Picanha Bovina,Embalagem,Geladeira,x6,1512.0,49.40,36.40,13.00,5.0
6,Miúdo de Porco,Embalagem,Geladeira,x7,13950.0,38.70,24.00,14.70,5.0
7,Pé de Frango,Caixa,Geladeira,x8,112404.0,85.00,34.00,51.00,0.0
8,Carré,Embalagem,Geladeira,x9,20160.0,81.12,48.00,33.12,8.0
9,Carne Moída,Embalagem,Geladeira,x10,15876.0,48.00,40.00,8.00,5.0


## Definição das variáveis 

In [2]:
volume = list(df['Volume [cm^3]'])
preco_venda = list(df['Venda (R$)'])
preco_compra = list(df['Compra'])
lucro = list(df['Lucro'])
demanda = list(df["Demanda"])

## Definição das matrizes de Entrada 

### Função Objetiva

Iremos minimizar a -f(x1, ... , x22), onde f(x1, ... , x22) é a função que representa o valor total da venda.

In [3]:
Z = [x * -1 for x in preco_venda]

### Restrições

Nesta etapa iremos impor as restrições das variáves.

#### Restrição Geladeira

Nesta etapa selecionaremos os itens que irão dentro da geladeira, ou seja, de x4 a x22.

In [4]:
aux = df.copy()
i = 0
for item in aux['Local transporte']:
    if item != 'Geladeira':
        aux.loc[i, 'Volume [cm^3]'] = 0
    i += 1
G = list(aux['Volume [cm^3]'])
max_G = 1369900

#### Restrição Kombi

Agora iremos impor as restrições para os itens que irão fora da geladeira. 

In [5]:
aux = df.copy()
i = 0
for item in aux['Local transporte']:
    if item == 'Geladeira':
        aux.loc[i, 'Volume [cm^3]'] = 0
    i += 1
K = list(aux['Volume [cm^3]'])
max_K = 1984000

#### Fluxo de caixa

Existe um limite diário de mercadória de R$ 6000,00.

In [6]:
F = [x for x in preco_compra]
max_F = 6000

# Aplicando o Simplex 

In [7]:
import scipy
from scipy.optimize import linprog

In [8]:
c = Z
A = [G, K, F]
B = [max_G, max_K, max_F]
x1_bd = (0, None)
x_bounds = (x1_bd,)*len(c)

In [9]:
modelo_simplex = linprog(c, A_ub=A, b_ub=B, bounds=x_bounds, method='Simplex', options={"disp":True})

Optimization terminated successfully.
         Current function value: -11288.382654
         Iterations: 8


In [13]:
modelo_simplex

     con: array([], dtype=float64)
     fun: -11288.382654466443
 message: 'Optimization terminated successfully.'
     nit: 8
   slack: array([2.32830644e-10, 1.98400000e+06, 9.09494702e-13])
  status: 0
 success: True
       x: array([  0.        ,   0.        ,   0.        , 806.87333599,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        , 175.94332405,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ])

In [11]:
c = Z
A = [G, K, F]
B = [max_G, max_K, max_F]
x_bounds = []
for item in demanda:
    x_bounds.append((round(item,0),None))

x_bounds = tuple(x_bounds)
x_bounds

((75.0, None),
 (100.0, None),
 (40.0, None),
 (15.0, None),
 (4.0, None),
 (5.0, None),
 (5.0, None),
 (0.0, None),
 (8.0, None),
 (5.0, None),
 (5.0, None),
 (3.0, None),
 (4.0, None),
 (0.0, None),
 (5.0, None),
 (5.0, None),
 (5.0, None),
 (10.0, None),
 (3.0, None),
 (4.0, None),
 (4.0, None),
 (4.0, None))

In [12]:
model_simplex = linprog(c, A_ub=A, b_ub=B, bounds=x_bounds, method='simplex', options={ 'disp': True})
model_simplex

Optimization terminated successfully.
         Current function value: -9342.765735
         Iterations: 9


     con: array([], dtype=float64)
     fun: -9342.765735471434
 message: 'Optimization terminated successfully.'
     nit: 9
   slack: array([ 0.00000000e+00,  1.37996800e+06, -9.09494702e-13])
  status: 0
 success: True
       x: array([ 75.        , 100.        ,  40.        , 171.69657925,
         4.        ,   5.        ,   5.        ,   0.        ,
         8.        ,   5.        ,   5.        ,   3.        ,
         4.        ,  72.62197264,   5.        ,   5.        ,
         5.        ,  10.        ,   3.        ,   4.        ,
         4.        ,   4.        ])